In [92]:
import pandas as pd

In [119]:
d_name = {"苏暮雨": "雨", "苏昌河": "河", "白鹤淮": "淮", "苏喆": "喆", "慕雨墨": "墨", "唐怜月": "月", "慕雪薇": "薇", "慕青羊": "羊"}
l_com = ["雨河", "雨淮", "雨喆", "雨墨", "雨月", "雨薇", "雨羊", "河淮", "淮喆", "墨月", "薇羊"]

df_role = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:D, J:Q, S", na_filter = False).dropna()
df_date = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:B, T:AD")

##### Check roles

In [ ]:
for index, row in df_role.iterrows():
    if row["角色"]:
        for name, aname in d_name.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                if row[aname] != 1:
                    print(row[0], row[1], aname, "should be 1")
            elif row[aname] == 1:
                print(row[0], row[1], aname, "should be 0")

##### Fill in roles

In [ ]:
for index, row in df_role.iterrows():
    if row["角色"]:
        for name, aname in d_name.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                df_role.loc[index, aname] = 1
            else:
                df_role.loc[index, aname] = 0

df_role.to_excel('ahz_fill_in.xlsx', engine='xlsxwriter', index=False)

print("===== fill in complete =====")

##### Count pages

In [148]:
df_count_page = pd.DataFrame()

pages_t = df_role[(df_role["组"]=="A") | (df_role["组"]=="B")]["页数"].sum()

for name, aname in d_name.items():
    # count "场数"
    df_count_page.loc["总", "场数"] = df_role[(df_role["组"]=="A") | (df_role["组"]=="B")]["页数"].count()
    df_count_page.loc[aname, "场数"] = df_role.value_counts(aname)[1]
    # count "页数"
    df_count_page.loc["总", "页数"] = pages_t
    pages_s = df_role[df_role[aname]==1]["页数"].sum()
    df_count_page.loc[aname, "页数"] = pages_s
    df_count_page.loc[aname, "占比"] = pages_s / pages_t
    # # count "A组"
    pages_a_t = df_count_page.loc["总", "A组"] = df_role[df_role["组"]=="A"]["页数"].sum()
    df_count_page.loc["总", "A%"] = pages_a_t / pages_t
    pages_a = df_role[(df_role["组"]=="A") & df_role[aname]==1]["页数"].sum()
    df_count_page.loc[aname, "A组"] = pages_a
    df_count_page.loc[aname, "A%"] = pages_a / pages_s

for name_ab in l_com:
    # count "场数"
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "场数"] = df_role[(df_role[name_ab[0]]==1) & (df_role[name_ab[1]]==1)]["页数"].count()
    # count "页数"
    pages_com = df_role[(df_role[name_ab[0]]==1) & (df_role[name_ab[1]]==1)]["页数"].sum()
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "页数"] = pages_com
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], "占比"] = pages_com / pages_t
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], name_ab[0] + "%"] = pages_com / df_count_page.loc[name_ab[0], "页数"]
    df_count_page.loc[name_ab[0] + "+" + name_ab[1], name_ab[1] + "%"] = pages_com / df_count_page.loc[name_ab[1], "页数"]

df_count_page = df_count_page.style.format({"场数": "{:,.0f}".format, "页数": "{:,.1f}".format, "占比": "{:,.2%}".format, "A组": "{:,.1f}".format, "A%": "{:,.2%}".format, 
                                            "雨%": "{:,.2%}".format, "河%": "{:,.2%}".format, "淮%": "{:,.2%}".format, "喆%": "{:,.2%}".format, "墨%": "{:,.2%}".format, "月%": "{:,.2%}".format, "薇%": "{:,.2%}".format, "羊%": "{:,.2%}".format})

df_count_page


,场数,页数,占比,A组,A%,雨%,河%,淮%,喆%,墨%,月%,薇%,羊%
总,512,323.1,nan%,163.6,50.63%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
雨,196,167.4,51.81%,129.1,77.12%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
河,106,99.8,30.89%,75.3,75.45%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
淮,100,80.1,24.79%,65.1,81.27%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
喆,43,44.8,13.87%,29.1,64.96%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
墨,57,41.9,12.97%,12.4,29.59%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
月,33,31.0,9.59%,5.2,16.77%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
薇,44,32.8,10.15%,19.3,58.84%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
羊,58,45.6,14.11%,28.6,62.72%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
雨+河,72,74.5,23.06%,nan,nan%,44.50%,74.65%,nan%,nan%,nan%,nan%,nan%,nan%


##### Count dates

In [62]:
dates = ["拍摄日期", "Unnamed: 20", "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25", "Unnamed: 26", "Unnamed: 27", "Unnamed: 28", "Unnamed: 29"]

for index, row in df_date.iterrows():
    if not pd.isna(row["拍摄日期"]):
        df_date.loc[index, "天数"] = 0
        for date in dates:
            if not pd.isna(df_date.loc[index, date]):
                df_date.loc[index, "天数"] += 1

df_count_date = df_date[["场次", "主要内容", "天数"]]
# df_count_date = df_count_date.fillna(0)
# df_count_date["天数"] = df_count_date["天数"].astype(int)
df_count_date = df_count_date.sort_values(by=["天数"], ascending=False)

# df_count_date.to_excel("ahz_count_date.xlsx", index=False)
df_count_date[:30]

,场次,主要内容,天数
186,10-12,抢夺眠龙,11.0
299,17-32,长剑碎裂,9.0
413,24-20b,剑无敌到,7.0
417,25-2,无敌入魔,7.0
157,8-13,送眠龙剑,6.0
158,8-15,阻止取剑,6.0
1,1-3,雨来营救,6.0
134,7-6,围攻暮雨,6.0
56,3-14,阴真逃走,6.0
132,7-3,剑客列阵,5.0
